In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2

In [3]:
html_data = requests.get("https://www.otomoto.pl/oferta/ferrari-f8-tributo-official-ferrari-dealer-ID6E2r6Z.html")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [4]:
eur = 4.57
print(eur)


4.57


In [5]:
def get_data(link):
    price = soup.find('span',attrs={"class":"offer-price__number"}).text
    price = price.replace(" ","")
    if "PLN" in price:
        price = price.replace("PLN","")
        price = float(price)
    elif "EUR" in price:
        price = price.replace("EUR","")
        price = float(price)
        price = price * eur
    else:
        price = None
    print(price)

In [6]:
price = soup.find('span',attrs={"class":"offer-price__number"}).text
price = price.replace(" ","")
if "PLN" in price:
    price = price.replace("PLN","")
    price = float(price)
elif "EUR" in price:
    price = price.replace("EUR","")
    price = float(price)
    price = price * eur
else:
    price = None
print(price)


1730700.1300000001


In [7]:
ID =  soup.find('div',attrs={"class":"offer-content__metabar"})
ID = ID.find('div')
ID = ID.find('span')[1].text
ID

KeyError: 1

In [ ]:
# def geturl(url):
#     html_data_url = requests.get(url)
#     soup_url = BeautifulSoup(html_data_url.content, 'html5lib')
#     location = soup.find('a',attrs={"class":"css-1ibwe9h e1nbpvi62"}).text
#     new_row = {}
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#             new_row = offer['href']

In [ ]:
html_data_url = requests.get("https://www.otomoto.pl/osobowe/")
soup_url = BeautifulSoup(html_data_url.content, 'html5lib')

# new_row = {}
# for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}).find_all('a',attrs={"class":"offer-title__link"}):
#     new_row = get_data(offer['href'])

In [ ]:
new_row = {}
for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
    if "https://www.otomoto.pl" in offer['href']:
        print(offer['href'])

In [ ]:
get_data("https://www.otomoto.pl/oferta/ferrari-f8-tributo-official-ferrari-dealer-ID6E2r6Z.html")

In [ ]:
print(new_row)

In [ ]:
Link_ERROR = 0
Read_announcements = 0
# page = 1
# while True:
    
for page in range(1,101):  
    try:
        html_data_link = requests.get(f"https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa?page={page}")
        page += 1
    except Exception:
        print("Link_ERROR")
        continue
    
    soup_link = BeautifulSoup(html_data_link.content, 'html5lib')
    
    if soup_link.find('div',attrs={"class":"css-pi2gf8 e1qwpsp45"}) is None:
        pass
    else:
        break

    for offer in soup_link.find_all('a',attrs={"data-cy":"listing-item-link"}):
            new_row = get_data("https://www.otodom.pl"+ offer['href'])
            
            
            if new_row == None:
                Link_ERROR += 1
            else:
                df_table = df_table.append(new_row, ignore_index=True)
                Read_announcements += 1
                clear_output(wait=True)
                print("Read_announcements ", Read_announcements)
                print("Link_ERROR ",Link_ERROR)     
                
print("Finish")